# CMPE 549: Bioinformatics, Fall 2023
## Assignment I -  Pairwise Sequence Global Alignment

Please describe your work clearly since this notebook is considered as your report.

Student: Büşra Oğuzoğlu
ID: 2022800036

In [192]:
import random
import requests
import numpy as np
import pandas as pd

from Bio.Align import substitution_matrices
from Bio import Align

### Needleman Wunsch (Global Sequence Alignment)

In [193]:
def needleman_wunsch_algorithm_linear_penalty(seq1, seq2, penalty_params):
    pass


def needleman_wunsch_algorithm_affine_penalty(seq1, seq2, penalty_params):
    pass

In [ ]:
"""
Implements the Needleman-Wunsch algorithm for global sequence alignment with a linear gap penalty.

This function aligns two sequences (seq1 and seq2) using dynamic programming. The alignment is global, meaning 
it spans the entire length of both sequences. The algorithm uses a linear gap penalty model, where each gap, 
regardless of its length, incurs a fixed penalty.

Parameters:
    seq1 (str): The first sequence to be aligned.
    seq2 (str): The second sequence to be aligned.
    penalty_params (dict): A dictionary containing the 'gap_penalty' value.
    scoring_matrix (DataFrame): A pandas DataFrame representing the scoring matrix (e.g., BLOSUM62) used for 
                                scoring matches and mismatches between elements of the sequences.

Returns:
    tuple: A tuple containing three elements:
        - The alignment score (float): The score of the optimal alignment.
        - Aligned sequence 1 (str): The first sequence with gaps ('-') inserted to achieve optimal alignment.
        - Aligned sequence 2 (str): The second sequence with gaps ('-') inserted to achieve optimal alignment.

The function first initializes a score matrix and a traceback matrix. The score matrix is filled based on the 
optimal scores calculated using the scoring matrix for matches/mismatches and the linear gap penalty for gaps.
The traceback matrix is used to reconstruct the optimal alignment path. The function returns the score of the 
optimal alignment and the aligned sequences themselves.
"""

In [194]:
import numpy as np

def needleman_wunsch_algorithm_linear_penalty(seq1, seq2, penalty_params, scoring_matrix):
    gap_penalty = penalty_params['gap_penalty']

    # Initialize matrices
    n, m = len(seq1), len(seq2)
    score_matrix = np.zeros((n+1, m+1))
    traceback_matrix = np.zeros((n+1, m+1), dtype=int)

    # Initialize first row and column
    for i in range(1, n+1):
        score_matrix[i][0] = score_matrix[i-1][0] - gap_penalty
    for j in range(1, m+1):
        score_matrix[0][j] = score_matrix[0][j-1] - gap_penalty

    # Fill score matrix and traceback matrix
    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = scoring_matrix.loc[seq1[i-1], seq2[j-1]]
            score_diagonal = score_matrix[i-1][j-1] + match_score
            score_up = score_matrix[i-1][j] - gap_penalty
            score_left = score_matrix[i][j-1] - gap_penalty

            score_matrix[i][j] = max(score_diagonal, score_up, score_left)

            # Traceback matrix: 1=diagonal, 2=up, 3=left
            if score_matrix[i][j] == score_diagonal:
                traceback_matrix[i][j] = 1
            elif score_matrix[i][j] == score_up:
                traceback_matrix[i][j] = 2
            else:
                traceback_matrix[i][j] = 3

    # Traceback to find alignment
    align1, align2 = '', ''
    i, j = n, m
    while i > 0 or j > 0:
        if traceback_matrix[i][j] == 1:
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 2:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1

    return score_matrix[n, m], align1, align2


In [ ]:
"""
Implements the Needleman-Wunsch algorithm for global sequence alignment with an affine gap penalty.

This function aligns two sequences (seq1 and seq2) using dynamic programming with an affine gap penalty model. 
The alignment is global, meaning it covers the entire length of both sequences. The affine gap penalty model 
differentiates between opening a new gap and extending an existing one, with different penalties for each.

Parameters:
    seq1 (str): The first sequence to be aligned.
    seq2 (str): The second sequence to be aligned.
    penalty_params (dict): A dictionary containing the 'gap_opening' and 'gap_extension' values, as well as the scoring matrix.
    
Returns:
    tuple: A tuple containing three elements:
        - The alignment score (float): The score of the optimal alignment.
        - Aligned sequence 1 (str): The first sequence with gaps ('-') inserted to achieve optimal alignment.
        - Aligned sequence 2 (str): The second sequence with gaps ('-') inserted to achieve optimal alignment.

The function uses three matrices: M (Match/Mismatch), I (Insertion), and D (Deletion). M tracks the scores 
of alignments without gaps, while I and D track scores for gaps in seq1 and seq2, respectively. These matrices 
are filled based on the optimal scores calculated using the provided scoring matrix for matches/mismatches and 
the affine gap penalties for opening and extending gaps.

The traceback process starts from the bottom-right of the matrices and proceeds to the top-left, constructing 
the optimal alignment by choosing the path that led to each cell's score. This involves deciding at each step 
whether the current cell in the alignment was reached by a match/mismatch, an insertion, or a deletion.
"""


In [195]:
def needleman_wunsch_algorithm_affine_penalty(seq1, seq2, penalty_params):
    gap_opening = penalty_params['gap_opening']
    gap_extension = penalty_params['gap_extension']
    scoring_matrix = penalty_params['scoring_matrix']

    n, m = len(seq1), len(seq2)
    M = np.zeros((n+1, m+1))
    I = np.full((n+1, m+1), -np.inf)
    D = np.full((n+1, m+1), -np.inf)
    traceback_matrix = np.zeros((n+1, m+1), dtype=int)

    # Initialize first row and column
    for i in range(1, n+1):
        M[i][0] = -np.inf
        D[i][0] = gap_opening + i * gap_extension
    for j in range(1, m+1):
        M[0][j] = -np.inf
        I[0][j] = gap_opening + j * gap_extension

    # Fill matrices
    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = scoring_matrix.loc[seq1[i-1], seq2[j-1]]

            # Calculate scores for M, I, D matrices
            M[i][j] = max(M[i-1][j-1], I[i-1][j-1], D[i-1][j-1]) + match_score
            I[i][j] = max(M[i][j-1] + gap_opening, I[i][j-1] + gap_extension)
            D[i][j] = max(M[i-1][j] + gap_opening, D[i-1][j] + gap_extension)

            # Determine traceback
            max_score = max(M[i][j], I[i][j], D[i][j])
            if max_score == M[i][j]:
                traceback_matrix[i][j] = 1
            elif max_score == I[i][j]:
                traceback_matrix[i][j] = 2
            else:
                traceback_matrix[i][j] = 3

    # Traceback to find alignment
    align1, align2 = '', ''
    i, j = n, m
    while i > 0 or j > 0:
        if traceback_matrix[i][j] == 1:
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 2:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1
        else:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1

    max_score = max(M[n][m], I[n][m], D[n][m])
    return max_score, align1, align2


### BLOSUM62 Scoring Matrix

Run the cell below to obtain the scoring matrix.

In [196]:
resp = requests.get(
    "https://raw.githubusercontent.com/rdpstaff/AlignmentTools/master/src/data/blosum62.txt"
)
_scoring_matrix = np.stack(
    [
        np.fromstring(line[2:].strip(), sep=" ")
        for line in resp.text.split("\n")
        if line and (line[0] not in ["#", " "])
    ]
)
_amino_acids = resp.text.split("\n")[6].strip().split("  ")


scoring_matrix = pd.DataFrame(
    _scoring_matrix, index=_amino_acids, columns=_amino_acids, dtype=int
)

scoring_matrix

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4
Q,-1,1,0,0,-3,5,2,-2,0,-3,...,-1,0,-1,-2,-1,-2,0,3,-1,-4
E,-1,0,0,2,-4,2,5,-2,0,-3,...,-1,0,-1,-3,-2,-2,1,4,-1,-4
G,0,-2,0,-1,-3,-2,-2,6,-2,-4,...,-2,0,-2,-2,-3,-3,-1,-2,-1,-4
H,-2,0,1,-1,-3,0,0,-2,8,-3,...,-2,-1,-2,-2,2,-3,0,0,-1,-4
I,-1,-3,-3,-3,-1,-3,-3,-4,-3,4,...,-3,-2,-1,-3,-1,3,-3,-3,-1,-4


### Generic Sequence Alignment Function

In [197]:
def align_sequences(seq1, seq2, penalty, penalty_params):
    if penalty == "linear":
        gap_penalty = penalty_params['gap_penalty']
        return needleman_wunsch_algorithm_linear_penalty(seq1, seq2, {'gap_penalty': gap_penalty}, penalty_params['scoring_matrix'])
    elif penalty == "affine":
        return needleman_wunsch_algorithm_affine_penalty(seq1, seq2, penalty_params)
    else:
        raise NotImplementedError


### Inputs

You can use the functions below to generate sample sequences to test your algorithms. 

In [198]:
aminoacids = [
    "A",
    "R",
    "N",
    "D",
    "C",
    "Q",
    "E",
    "G",
    "H",
    "I",
    "L",
    "K",
    "M",
    "F",
    "P",
    "S",
    "T",
    "W",
    "Y",
    "V",
]


def generate_sequence(length=50):
    return "".join([random.choice(aminoacids) for i in range(length)])


def mutate_sequence(seq, n_mutations=10):
    seq = list(seq)
    pos = {
        random.randint(1, len(seq)): random.choice(["substitute", "delete"])
        for i in range(n_mutations)
    }
    mutated_sequence = ""
    for ix, aminoacid in enumerate(seq):
        if ix in pos:
            if pos[ix] == "substitute":
                mutated_sequence += random.choice(aminoacids)
        else:
            mutated_sequence += aminoacid
    return mutated_sequence


sample_sequence = generate_sequence()
sequence1 = mutate_sequence(sample_sequence)
sequence2 = mutate_sequence(sample_sequence)
sequence1, sequence2

('WNIKSNLAFTRKHRVAVIFVNLILVALWHYQHHLAGIELGKSYKLYFF',
 'WKNYLAFTRKHRKAWIFVNLILEALDGHYQPHHLSGIELKSYKAYFF')

#### Sample 1

In [199]:
sample1_sequence1 = "PLEASANTLY"
sample1_sequence2 = "MEANLY"

In [200]:
def align_sequences_biopython(seq1, seq2, penalty_params):
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    if penalty_params["penalty_type"] == "linear":
        aligner.open_gap_score = penalty_params["gap_penalty"]  # -1
        aligner.extend_gap_score = penalty_params["gap_penalty"]  # -1
    elif penalty_params["penalty_type"] == "affine":
        aligner.open_gap_score = penalty_params["gap_penalty"]  # -10
        aligner.extend_gap_score = penalty_params["gap_extension_penalty"]  # -1

    aligner.mode = "global"
    for alignment in aligner.align(seq1, seq2):
        print("Score:", alignment.score)
        print(alignment)

In [201]:
align_sequences_biopython(
    sample1_sequence1, sample1_sequence2, {"penalty_type": "linear", "gap_penalty": -1}
)

Score: 24.0
target            0 PLEASANTLY 10
                  0 -.||--|-|| 10
query             0 -MEA--N-LY  6

Score: 24.0
target            0 PLEASANTLY 10
                  0 -.|--||-|| 10
query             0 -ME--AN-LY  6



In [202]:
align_sequences_biopython(
    sample1_sequence1,
    sample1_sequence2,
    {"penalty_type": "affine", "gap_penalty": -10, "gap_extension_penalty": -1},
)

Score: 1.0
target            0 PLEASANTLY 10
                  0 -.||.---|| 10
query             0 -MEAN---LY  6



#### Sample 2

In [203]:
sample2_sequence1 = "PRTEINS"
sample2_sequence2 = "PRTWPSEIN"

In [204]:
align_sequences_biopython(
    sample2_sequence1, sample2_sequence2, {"penalty_type": "linear", "gap_penalty": -1}
)

Score: 28.0
target            0 PRT---EINS  7
                  0 |||---|||- 10
query             0 PRTWPSEIN-  9



In [205]:
align_sequences_biopython(
    sample2_sequence1,
    sample2_sequence2,
    {"penalty_type": "affine", "gap_penalty": -10, "gap_extension_penalty": -1},
)

Score: 10.0
target            0 PRT---EINS  7
                  0 |||---|||- 10
query             0 PRTWPSEIN-  9



#### Sample 3

In [206]:
sample3_sequence1 = (
    "YHFDVPDCWAHRYWVENPQAIAQMEQICFNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE"
)
sample3_sequence2 = (
    "YHEDVAHEDAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE"
)

### Your Output

Please run the cells below to present your final output.

#### Test Case 1

In [207]:
seq1 = "PRTEINS"
seq2 = "PRTWPSEIN"

In [208]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1, seq2, "linear", {"scoring_matrix": scoring_matrix, "gap_penalty": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)


36.0
PRT---EINS
PRTWPSEIN-


In [209]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1,
    seq2,
    "affine",
    {"scoring_matrix": scoring_matrix, "gap_opening": -10, "gap_extension": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

10.0
PRT---EINS
PRTWPSEIN-


#### Test Case 2

In [210]:
seq1 = "QMETSTTGAAYFNG"
seq2 = "QFISPTGAAKYFNT"

In [211]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1, seq2, "linear", {"scoring_matrix": scoring_matrix, "gap_penalty": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

57.0
Q--METS-TTGAA-YFN-G
QFI---SP-TGAAKYFNT-


In [212]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1,
    seq2,
    "affine",
    {"scoring_matrix": scoring_matrix, "gap_opening": -10, "gap_extension": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

25.0
QMETSTTGAAYFNG
QFISPTGAAKYFNT


#### Test Case 3

In [213]:
seq1 = "DAHGDMWNYFRANETVAWEWFPHHAQFLHCELPKDK"
seq2 = "DAHGGCDWNYFANELTVMWEHFNPHAQFLHCELEYMDL"

In [214]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1, seq2, "linear", {"scoring_matrix": scoring_matrix, "gap_penalty": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

190.0
DAH-G-DMWNYFRANE-TV-AWE-WF-PHHAQFLHCEL---PKD-K
DAHGGCD-WNYF-ANELTVM-WEH-FNP-HAQFLHCELEYM--DL-


In [215]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1,
    seq2,
    "affine",
    {"scoring_matrix": scoring_matrix, "gap_opening": -10, "gap_extension": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

106.0
DAHGDM-WNYFRANE-TVAWEWFPHHAQFLHCELP-KDK
DAHGGCDWNYF-ANELTVMWEHFNPHAQFLHCELEYMDL


#### Test Case 4

In [216]:
seq1 = "SKGSLWLWMCKDKLGENLVVFNNVLGAGLFDEPHYPCVEWYDFVR"
seq2 = "SHKGSLNWLGWNCKDKLGENLVVFPNYLFAAHFEPHYPCIVEYDFVD"

In [217]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1, seq2, "linear", {"scoring_matrix": scoring_matrix, "gap_penalty": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

228.0
S-KGSL-WL-W-MCKDKLGENLVVF-N-NVL--GA-GLFDEPHYPC-VEWYDFV-R
SHKGSLNWLGWN-CKDKLGENLVVFPNY--LFA-AH--F-EPHYPCIVE-YDFVD-


In [218]:
alignment_score, aligned_seq1, aligned_seq2 = align_sequences(
    seq1,
    seq2,
    "affine",
    {"scoring_matrix": scoring_matrix, "gap_opening": -10, "gap_extension": -1}
)
print(alignment_score)
print(aligned_seq1)
print(aligned_seq2)

144.0
S-KGSL-WL-WMCKDKLGENLVVFNNVLGAGLFDEPHYPCVEWYDFVR
SHKGSLNWLGWNCKDKLGENLVVFPNYLFAAHF-EPHYPCIVEYDFVD
